In [1]:
!pip install --upgrade pip

!pip install git+https://github.com/huggingface/transformers.git accelerate
!pip install torch torchvision torchaudio

  Obtaining dependency information for pip from https://files.pythonhosted.org/packages/47/6a/453160888fab7c6a432a6e25f8afe6256d0d9f2cbd25971021da6491d899/pip-23.3.1-py3-none-any.whl.metadata
  Using cached pip-23.3.1-py3-none-any.whl.metadata (3.5 kB)
Using cached pip-23.3.1-py3-none-any.whl (2.1 MB)
  Attempting uninstall: pip
    Found existing installation: pip 23.2.1
    Uninstalling pip-23.2.1:
      Successfully uninstalled pip-23.2.1
  Cloning https://github.com/huggingface/transformers.git to /private/var/folders/1q/n_w7wck159d6t0_lt311ssmc0000gn/T/pip-req-build-jko00t6j
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /private/var/folders/1q/n_w7wck159d6t0_lt311ssmc0000gn/T/pip-req-build-jko00t6j
  Resolved https://github.com/huggingface/transformers.git to commit d1a00f9dd0b851245e4a54cbd70816a80e781ec2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.

In [2]:
!pip install ffmpeg-python

In [ ]:
import torch
import json
import datetime

from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline

# device = "cuda:0" if torch.cuda.is_available() else "cpu"
device ="cpu"
# torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32
torch_dtype=torch.float32

model_id = "openai/whisper-large-v3"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=30,
    batch_size=16,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=device,
    generate_kwargs={"language": "cantonese"}
)


# this is the place you modify your input - the name of the mp3 file you want to run
result = pipe("sample.mp3")

# then it will write the response in a json file named as the current date time
now = datetime.datetime.now().strftime("%d-%m-%Y-%H-%M-%S")
json_object = json.dumps(result, indent=4)
with open(now+".json", "w") as f:
    f.write(json_object)

# also it will print out the result in the following output block
print(result)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
